In [1]:
import time
import json
import pandas as pd
from tqdm import tqdm
from openai import OpenAI


client = OpenAI(api_key="api privada")


df = pd.read_csv("dados limpos.csv", sep=None, engine="python", encoding="utf-8-sig")
df = df.sample(50).copy()
df["velocidade_estim_kmh"] = None  # nova coluna


SYSTEM_PROMPT = """Você é um perito de trânsito no Brasil.
Sua tarefa é estimar, de forma plausível, a velocidade aproximada (em km/h) do veículo principal envolvido no acidente,
usando apenas os seguintes campos: causa_acidente, tipo_acidente, classificacao_acidente, mortos, feridos_leves, feridos_graves, ilesos, veiculos.

Regras e limitações:
- Não invente fatos além do que está nos campos fornecidos.
- Responda estritamente com um JSON válido e nada mais.

Formato de saída obrigatório:
{
  "velocidade_kmh": <int|null>
}
"""

USER_PROMPT_TEMPLATE = """Dados do acidente:
- causa_acidente: {causa_acidente}
- tipo_acidente: {tipo_acidente}
- classificacao_acidente: {classificacao_acidente}
- mortos: {mortos}
- feridos_leves: {feridos_leves}
- feridos_graves: {feridos_graves}
- ilesos: {ilesos}
- veiculos: {veiculos}

Tarefa:
- Estime "velocidade_kmh" (número inteiro, em km/h) do veículo principal no momento do acidente.
- Responda estritamente no JSON especificado, sem texto adicional.
"""


for idx, row in tqdm(df.iterrows(), total=len(df), desc="Estimando velocidades", unit="linha"):
    causa   = str(row["causa_acidente"]).strip()
    tipo    = str(row["tipo_acidente"]).strip()
    classif = str(row["classificacao_acidente"]).strip()

    mortos         = int(row["mortos"])
    feridos_leves  = int(row["feridos_leves"])
    feridos_graves = int(row["feridos_graves"])
    ilesos         = int(row["ilesos"])
    veiculos       = int(row["veiculos"])

    prompt = USER_PROMPT_TEMPLATE.format(
        causa_acidente=causa,
        tipo_acidente=tipo,
        classificacao_acidente=classif,
        mortos=mortos,
        feridos_leves=feridos_leves,
        feridos_graves=feridos_graves,
        ilesos=ilesos,
        veiculos=veiculos,
    )

    try:
        resp = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user",   "content": prompt},
            ],
            temperature=0.2,
            response_format={"type": "json_object"},
        )
        content = resp.choices[0].message.content
        data = json.loads(content)
        val = data.get("velocidade_kmh", None)
        speed = None if val is None else int(round(float(val)))
    except Exception:
        speed = None

    df.at[idx, "velocidade_estim_kmh"] = speed
    time.sleep(0.3)  # pausa leve para evitar rate-limit

# === 5) Salvar CSV final ===
df.to_csv("dados limpos com kmh.csv", index=False, encoding="utf-8-sig")
print("✅ Exportado: dados limpos com kmh.csv")

Estimando velocidades: 100%|██████████| 50/50 [00:54<00:00,  1.09s/linha]

✅ Exportado: dados limpos com kmh.csv


In [7]:
df_new = pd.read_csv('dados limpos com kmh.csv')
pd.set_option('display.max_columns', None)
df_new.sample(50)

,id,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,uso_solo,pessoas,mortos,feridos_leves,feridos_graves,ilesos,ignorados,feridos,veiculos,latitude,longitude,regional,delegacia,uop,ano,mes,dia,fim_de_semana,estacao,velocidade_estim_kmh
4,503138,2023-02-04,sabado,19:10:00,RJ,493,106.0,DUQUE DE CAXIAS,Demais falhas mecanicas ou eletricas,Saida de leito carrocavel,Com Vitimas Feridas,Pleno dia,Decrescente,Ceu Claro,Dupla,Reta,Nao,3,0,2,0,1,0,2,1,-22.672089,-43.285053,SPRF-RJ,DEL03-RJ,UOP01-DEL03-RJ,2023,2,4,True,Verão,30
33,418193,2022-03-02,quarta-feira,15:00:00,RO,364,703.0,PORTO VELHO,Manobra de mudanca de faixa,Colisao lateral mesmo sentido,Sem Vitimas,Pleno dia,Crescente,Ceu Claro,Dupla,Reta,Sim,5,0,0,0,5,0,0,2,-8.800215,-63.795578,SPRF-RO,DEL01-RO,UOP01-DEL01-RO,2022,3,2,False,Verão,30
13,397871,2021-11-25,quinta-feira,15:40:00,GO,50,273.0,CATALAO,Velocidade Incompativel,Tombamento,Com Vitimas Feridas,Pleno dia,Crescente,Ceu Claro,Simples,Viaduto,Sim,2,0,1,0,0,3,1,4,-18.130200,-47.904700,SPRF-GO,DEL06-GO,UOP01-DEL06-GO,2021,11,25,False,Primavera,60
0,548745,2023-08-23,quarta-feira,11:40:00,MG,116,799.2,ALEM PARAIBA,Ausencia de reacao do condutor,Colisao traseira,Com Vitimas Fatais,Pleno dia,Decrescente,Ceu Claro,Simples,Reta;Em Obras,Nao,3,1,0,0,1,2,0,4,-21.753440,-42.732397,SPRF-MG,DEL07-MG,UOP01-DEL07-MG,2023,8,23,False,Inverno,50
43,568562,2023-12-17,domingo,17:10:00,BA,116,285.0,TUCANO,Ingestao de alcool pelo condutor,Colisao frontal,Com Vitimas Feridas,Pleno dia,Decrescente,Ceu Claro,Simples,Reta,Sim,3,0,0,1,1,1,1,3,-11.047720,-38.838037,SPRF-BA,DEL02-BA,UOP01-DEL02-BA,2023,12,17,True,Primavera,60
34,504779,2023-02-12,domingo,19:50:00,MG,381,531.1,ITATIAIUCU,Chuva,Saida de leito carrocavel,Com Vitimas Feridas,Plena Noite,Crescente,Chuva,Dupla,Curva;Declive,Nao,2,0,2,0,0,0,2,1,-20.163288,-44.358673,SPRF-MG,DEL04-MG,UOP03-DEL04-MG,2023,2,12,True,Verão,40
27,558964,2023-10-23,segunda-feira,00:20:00,SE,101,173.3,SANTA LUZIA DO ITANHY,Ingestao de alcool pelo condutor,Colisao traseira,Com Vitimas Feridas,Plena Noite,Decrescente,Ceu Claro,Simples,Curva,Nao,6,0,2,0,4,0,2,2,-11.339671,-37.594680,SPRF-SE,DEL02-SE,UOP01-DEL02-SE,2023,10,23,False,Primavera,30
5,466241,2022-08-06,sabado,05:05:00,MG,116,556.0,MANHUACU,Transitar na contramao,Colisao frontal,Com Vitimas Fatais,Amanhecer,Crescente,Ceu Claro,Simples,Reta,Nao,49,1,1,0,47,0,1,2,-20.009515,-42.151780,SPRF-MG,DEL06-MG,UOP02-DEL06-MG,2022,8,6,True,Inverno,80
19,500533,2023-01-21,sabado,07:40:00,GO,414,439.0,ANAPOLIS,Acessar a via sem observar a presenca dos outr...,Colisao transversal,Com Vitimas Feridas,Pleno dia,Decrescente,Nublado,Simples,Intersecao de Vias;Reta,Sim,2,0,1,0,1,0,1,2,-16.281288,-48.942589,SPRF-GO,DEL02-GO,UOP01-DEL02-GO,2023,1,21,True,Verão,30
38,272685,2020-03-01,domingo,20:10:00,MG,50,93.8,UBERLANDIA,Pista Escorregadia,Saida de leito carrocavel,Com Vitimas Feridas,Plena Noite,Decrescente,Chuva,Dupla,Aclive,Nao,1,0,1,0,0,0,1,1,-19.074925,-48.188923,SPRF-MG,DEL15-MG,UOP01-DEL15-MG,2020,3,1,True,Verão,30
